before start the testing, see Readme, be sure the IPbus, control Hub and python 3.8 is installed.
To properly set all paths run source setup.sh.

## Cell 1: KCU Connection Setup
This cell establishes connection to the KCU board and verifies basic communication.
- Sets KCU IP address to 192.168.0.10
- Enables control hub mode
- Performs firmware version check and loopback test

**Expected Output:** 
- "Successfully connected to KCU"
- Firmware version information
- "KCU Loopback test PASSED"


In [ ]:
from tamalero.utils import get_kcu

kcu_ip = "192.168.0.10" ## If your KCU ip is diff, modify it.

kcu = get_kcu(
    kcu_ip,
    control_hub=True,
    host='localhost',
    verbose=False
)
print("Successfully connected to KCU.")

# Check the KCU's status and firmware
#kcu.status() # Prints LpGBT link statuses from KCU 
fw_ver = kcu.get_firmware_version(verbose=True) #
kcu.check_clock_frequencies(verbose=True) # Verifies KCU clock stability

# Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()
if read_val == loopback_val:
    print(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")
else:
    print(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")

This cell configures the ReadoutBoard object and establishes lpGBT communication.

Steps performed:
1. Initialize ReadoutBoard with modulev1 configuration
2. Detect and verify readout board version
3. Check lpGBT link status
4. Verify no FEC errors on DAQ link

Troubleshooting:
- If version detection fails, check lpGBT connection
- If FEC errors > 0, reset error counters and check links


In [ ]:
from tamalero.ReadoutBoard import ReadoutBoard

rb = ReadoutBoard(
    rb=0,
    kcu=kcu,
    config="modulev1", 
    trigger=False,     
    verbose=True
)

print(f"Readout Board version detected: {rb.ver}")

- If FEC errors > 0, no worries, READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT was reset later
READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT is red due to our current setup only have one uplink which is uplink 0

In [ ]:
print("Checking DAQ LpGBT base configuration:")
rb.DAQ_LPGBT.read_base_config() #

# Read on-board temperatures
print("\nReading temperatures:")
rb.read_temp(verbose=True) #

print("\nReading DAQ LpGBT ADCs:")
try:
    rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
except ValueError as e:
    print(f"LpGBT ADC check issue: {e}")


print(f"\nDAQ LpGBT Link Good: {rb.DAQ_LPGBT.link_status()}") 
fec_errors = rb.get_FEC_error_count() # Reads KCU registers defined in READOUT_BOARD.xml
print(f"FEC Errors: DAQ LpGBT = {fec_errors.get('DAQ', 'N/A')}")



Init ETROC2, our etroc2 i2c address is 0x63 and i2c channel is 1, modify it if yours diff

In [ ]:
from tamalero.ETROC import ETROC
from tamalero.colors import green, red, yellow
from tamalero.LPGBT import LPGBT
import time

etroc_elinks_map = {0: [0, 4, 8, 12]} # should map your design
print("Initializing ETROC objects...")
my_etroc2s = [ 
    ETROC(
    rb,
    master='lpgbt',
    i2c_adr=0x63,
    chip_id=63,                 
    i2c_channel=1,
    elinks=etroc_elinks_map,
    strict=False,
    verbose=True,
    ),
    ETROC(
    rb,
    master='lpgbt',
    i2c_adr=0x62,
    chip_id=62,                 
    i2c_channel=1,
    elinks=etroc_elinks_map,
    strict=False,
    verbose=True 
    ),
    ETROC(
    rb,
    master='lpgbt',
    i2c_adr=0x61,
    chip_id=61,                  
    i2c_channel=1,
    elinks=etroc_elinks_map,
    strict=False,
    verbose=True 
    ),
    ETROC(
    rb,
    master='lpgbt',
    i2c_adr=0x60,
    chip_id=60,                 
    i2c_channel=1,
    elinks=etroc_elinks_map,
    strict=False,
    verbose=True 
    ),
]

In [ ]:
for my_etroc2 in my_etroc2s:
    print("------------Board------------")
    if not my_etroc2.is_connected():
        raise ConnectionError("Failed to connect to ETROC2 via I2C.")

    print(green("SUCCESS: I2C communication with ETROC2 is established."))

    print("-" * 50)
    print("Verifying a key configuration register...")
    register_to_check = 'disScrambler'
    expected_value = 1

    read_back_value = my_etroc2.rd_reg(register_to_check)
    print(f"Reading back '{register_to_check}' register...")
    print(f"  - Value Expected: {expected_value}")
    print(f"  - Value Read Back: {read_back_value}")

    if read_back_value == expected_value:   ## check etroc register can be read correctly
        print(green("  - SUCCESS: Register read-back matches expected value."))
    else:
        print(red("  - FAILURE: Register read-back does NOT match expected value."))

Just checking etroc status and see which elink is locked.
if elink is unlock, can't retrieve data from etroc

In [ ]:
for my_etroc2 in my_etroc2s:
    print("------------Board------------")
    
    print(f"ETROC connected: {my_etroc2.is_connected()}")
    print(f"ETROC version: {my_etroc2.get_ver()}")

    # Check if ETROC is in a good state
    if hasattr(my_etroc2, 'is_good'):
        print(f"ETROC is good: {my_etroc2.is_good()}")

    # Check controller state
    if hasattr(my_etroc2, 'controllerState'):
        state = my_etroc2.rd_reg("controllerState")
        print(f"Controller state: {state} (should be 11)")

    # Check Pll lock status
    Pll_state = my_etroc2.rd_reg("pllUnlockCount")
    print(f"PLL Unlock Count: {Pll_state} (should be 0 or very few and would not increase)")

    # Check elink lock status
    print("\n=== Elink Status ===")
    elink_status = my_etroc2.get_elink_status()
    print(f"Elink status: {elink_status}")


    for elink in [0, 4, 8, 12]: 
        locked = rb.etroc_locked(elink, slave=False)
        print(f"Elink {elink} locked: {locked}")
        
        if locked:
            packet_count = rb.read_packet_count(elink, slave=False)
            error_count = rb.read_error_count(elink, slave=False)
            filler_rate = rb.read_filler_rate(elink, slave=False)
            
            print(f"  Packets: {packet_count}")
            print(f"  Errors: {error_count}")
            print(f"  Filler rate: {filler_rate}")
            
    print('-' * 50)
    eprx_group_to_check = [0,1,2,3]
        
    try:
        for group in eprx_group_to_check:
            # Read the Read-Only status register
            dll_locked_status = rb.DAQ_LPGBT.rd_reg(f"LPGBT.RO.EPORTRX_RO.EPRX.EPRX{group}DLLLOCKED")
            if dll_locked_status == 1:
                print(green(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (Locked)"))
            else:
                print(red(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (NOT Locked)"))
    except Exception as e:
        print(red(f"dddCould not read LpGBT EPRX DLL status. Error: {e}"))


check uplink status and rese FEC errors if not 0

In [ ]:
print("=== Check Uplink Status ===")

# Check uplink error counts
uplink0_errors = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.FEC_ERR_CNT").value()

print(f"Uplink 0 FEC errors: {uplink0_errors}")

# Check uplink ready status
uplink0_ready = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.READY").value()

print(f"Uplink 0 ready: {uplink0_ready}")

# Reset FEC error counters
print("\nResetting FEC error counters...")
rb.reset_FEC_error_count()

print("\nChecking elink 12 status again...")
rb.reset_data_error_count()
time.sleep(1)

for elinks in [0, 4, 8, 12]:  # we current only have elink 12 locked
    locked = rb.etroc_locked(12, slave=False)
    packet_count = rb.read_packet_count(elinks, slave=False)
    error_count = rb.read_error_count(elinks, slave=False)
    filler_rate = rb.read_filler_rate(elinks, slave=False)

    print(f"Elink {elinks} - Locked: {locked}, Packets: {packet_count}, Errors: {error_count}, Filler: {filler_rate}")

    if filler_rate > 15000000 and error_count < 100:
        print(f"Elink {elinks} appears to be working!")
    else:
        print(f"Elink {elinks} still has issues")

just make sure lpGBT assignment is correct

In [ ]:
print("=== Verify lpGBT Assignment ===")

print("Checking master vs slave elink locks...")

# Check master elinks (lpgbt=0)
master_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED").value()
print(f"Master (lpgbt=0) elinks locked: {bin(master_locked)}")

# Check slave elinks (lpgbt=1) 
slave_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED_SLAVE").value()
print(f"Slave (lpgbt=1) elinks locked: {bin(slave_locked)}")

for elink in [0, 4, 8, 12, 14]:
    print(f"\nElink {elink}:")

    locked_master = rb.etroc_locked(elink, slave=False)
    print(f"  lpgbt=0: {locked_master}")
    
    locked_slave = rb.etroc_locked(elink, slave=True)
    print(f"  lpgbt=1: {locked_slave}")
    
    if locked_master:
        print(f"  → Elink {elink} is on lpgbt=0 (DAQ)")
    elif locked_slave:
        print(f"  → Elink {elink} is on lpgbt=1 (Trigger)")
    else:
        print(f"  → Elink {elink} not locked on either lpGBT")

for my_etroc2 in my_etroc2s:
    print("------------Board------------")
    print(f"\n2. Your ETROC elinks configuration:")
    print(f"elinks[0]: {my_etroc2.elinks.get(0, 'Not configured')}")
    print(f"elinks[1]: {my_etroc2.elinks.get(1, 'Not configured')}")

try to reproduce data from ETROC without charge injection
under workmode : 0(normal mode) data type should be only contain header and trailer only

In [ ]:
for my_etroc2 in my_etroc2s:
    print("------------Board------------")
    my_etroc2.set_power_mode(mode="high", row=0, col=0, broadcast=True)
    my_etroc2.run_threshold_scan(offset=15, use=True, out_dir=None)

In [ ]:
for my_etroc2 in my_etroc2s:
    print("------------Board------------")
    my_etroc2.plot_threshold(outdir='results/', noise_width=True)
    my_etroc2.plot_threshold(outdir='results/', noise_width=False)

In [ ]:
clks = [0,0,0,0]
datas = [0,0,0,0]
invs = [0,0,1,0]
for i,my_etroc2 in enumerate(my_etroc2s):
    print("------------Board------------")
    print(i, my_etroc2.FC_status(), my_etroc2.get_invalidFCCount())
    if(invs[i]):
        my_etroc2.wr_reg('FC_InvData', 1)
    else:
        my_etroc2.wr_reg('FC_InvData', 0)

    my_etroc2.reset_fast_command()
    
    # my_etroc2.wr_reg('asyAlignFastcommand', 0)
    # if(clks[i]):
    #     my_etroc2.enable_fcClkDelay()
    # else:
    #     my_etroc2.disable_fcClkDelay()
    # if(datas[i]):
    #     my_etroc2.enable_fcDataDelay()
    # else:
    #     my_etroc2.disable_fcDataDelay()
    print(i, my_etroc2.FC_status(), my_etroc2.get_invalidFCCount())
    print(i, my_etroc2.FC_status(), my_etroc2.get_invalidFCCount())

In [ ]:
for i,my_etroc2 in enumerate(my_etroc2s):
    print("------------Board------------")
    print(i, my_etroc2.FC_status(), my_etroc2.get_invalidFCCount())
    print(i, my_etroc2.FC_status(), my_etroc2.get_invalidFCCount())

In [ ]:
from tamalero.FIFO import FIFO
from tamalero.DataFrame import DataFrame

print("\n=== Get data with charge injection ===")
df = DataFrame()
fifo = FIFO(rb)

fifo.reset()
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()
time.sleep(1)

delays = [501, 501, 501, 501]
pixels_to_test = [(12, 8)]

rb.DAQ_LPGBT.set_uplink_group_data_source("normal") 
for i,my_etroc2 in enumerate(my_etroc2s):
    my_etroc2.reset()
    my_etroc2.disable_QInj(broadcast=True)
    my_etroc2.disable_data_readout(broadcast=True)
    my_etroc2.disable_trigger_readout(broadcast=True)
    for (pixel_row,pixel_col) in pixels_to_test:
        my_etroc2.QInj_set(charge=30, delay=10, L1Adelay=delays[i], row=pixel_row, col=pixel_col, broadcast=False, reset=True)
    pixels_to_test = pixels_to_test + [(pixels_to_test[-1][0]+1,pixels_to_test[-1][1]+1)]

print(green("Configuration complete."))
# fifo.send_l1a(4096)  ## l1a value based on test_ETROC.py
fifo.send_QInj(1000, delay=my_etroc2.QINJ_delay)

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    if occupancy > 0:
        print(green("SUCCESS: Data is being generated!"))
        print(f"   FIFO returned {occupancy} data items")

        for j, word in enumerate(data[:24]):
            data_type, event_data = word[0], word[1]
            print(f"  Event {j}: {data_type} -> {event_data}")
           
except Exception as e:
    print(red(f"Read failed: {e}"))

finally:
    print("\nCleaning up...")
    for i,my_etroc2 in enumerate(my_etroc2s):
        my_etroc2.disable_QInj(broadcast=True)
        my_etroc2.disable_data_readout(broadcast=True)
        my_etroc2.disable_trigger_readout(broadcast=True)
    print("Test complete!")


# fifo.set_trigger_rate(1000)
# time.sleep(5)
# fifo.set_trigger_rate(0)


# occupancy = fifo.get_occupancy()
# lost_words = fifo.get_lost_word_count()

# print(f"Occupancy after continuous trigger: {occupancy}")
# print(f"Lost words: {lost_words}")
# time.sleep(1)


# if occupancy > 0 or lost_words > 0:
#     print("SUCCESS: Data is being generated!")
#     try:
#         # Use the simple read method
#         raw_data = fifo.read(dispatch=True)
#         print(f"✅ Successfully read {len(raw_data)} words!")
            
#         if len(raw_data) >= 2:
#             from tamalero.FIFO import merge_words
#             merged = merge_words(raw_data[:20]) 
                
#             print(f"Merged {len(merged)} events:")
#             for j, word in enumerate(merged[:10]):
#                 data_type, event_data = df.read(word)
                
#                 print(f"  Event {j}: {data_type} -> {event_data}")
            
#     except Exception as e:
#         print(f"Read failed: {e}")

    

In [ ]:
import os, struct
charge = 30
pixels_to_test = [(12, 8)]
output_dir = "Qinj_Output"
os.makedirs(output_dir, exist_ok=True)
output_dat_file = os.path.join(output_dir, f"qinj_{charge}fC.dat")

print("=== Charge Injection Test ===")
print(f"Charge: {charge} fC")
print(f"Saving raw data to: {output_dat_file}")

# Enable readout only for the pixel under test
# Set charge injection parameters
delays = [501, 501, 501, 501]
for i,my_etroc2 in enumerate(my_etroc2s):
    my_etroc2.reset()
    my_etroc2.disable_QInj(broadcast=True)
    my_etroc2.disable_data_readout(broadcast=True)
    my_etroc2.disable_trigger_readout(broadcast=True)
    for (pixel_row,pixel_col) in pixels_to_test:
        my_etroc2.QInj_set(charge=30, delay=10, L1Adelay=delays[i], row=pixel_row, col=pixel_col, broadcast=False, reset=True)
    pixels_to_test = pixels_to_test + [(pixels_to_test[-1][0]+1,pixels_to_test[-1][1]+1)]
# # The QSel register takes a value from 0-31 to represent 1-32 fC
# my_etroc2.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col)
# my_etroc2.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col) #enable charge injection
print(green("Configuration complete."))

fifo = FIFO(rb)
fifo.reset()
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()
time.sleep(1)

try:
    fifo.send_QInj(500, delay=my_etroc2.QINJ_delay)
    raw_data = fifo.read(dispatch=True)
    print(green(f"✅ Successfully read {len(raw_data)} words from the FIFO."))

    # --- Save Raw Data ---
    with open(output_dat_file, "wb") as f:
        f.write(struct.pack('<{}I'.format(len(raw_data)), *raw_data))
    print(green(f"✅ Raw data saved to {output_dat_file}"))

except Exception as e:
    print(red(f"An error occurred: {e}"))

finally:
    # --- Cleanup ---
    print("\nCleaning up...")
    for i,my_etroc2 in enumerate(my_etroc2s):
        my_etroc2.disable_QInj(broadcast=True)
        my_etroc2.disable_data_readout(broadcast=True)
        my_etroc2.disable_trigger_readout(broadcast=True)
    print("Test complete!")

In [ ]:
import struct
import os
import numpy as np
import pandas as pd
import awkward as ak
import json
from tamalero.DataFrame import DataFrame
from tamalero.colors import green, red

def merge_words(res):
    empty_frame_mask = np.array(res[0::2]) > (2**8)
    len_cut = min(len(res[0::2]), len(res[1::2]))
    if len(res) > 0:
        return list(np.array(res[0::2])[:len_cut][empty_frame_mask[:len_cut]] | (np.array(res[1::2]) << 32)[:len_cut][empty_frame_mask[:len_cut]])
    else:
        return []

def _aggregate_fragments(fragments):
    """
    Helper function to combine a list of event fragments into a single event record.
    This version now correctly handles cases where trailers (and chipid) are missing.
    """
    df = pd.DataFrame(fragments)
    
    headers = df[df['l1counter'].notna()]
    hits = df[df['toa'].notna()]
    
    # Robustly handle missing trailers
    if 'chipid' in df.columns:
        trailers = df[df['chipid'].notna()]
    else:
        # If no trailers exist in this fragment, create an empty DataFrame
        trailers = pd.DataFrame()

    all_hits = {
        'row_id': [], 'col_id': [], 'toa_code': [], 'tot_code': [],
        'cal_code': [], 'elink': [], 'chip_id': []
    }

    for elink in hits['elink'].unique():
        elink_hits = hits[hits['elink'] == elink]
        
        # Only look for trailers if the trailers DataFrame is not empty
        if not trailers.empty:
            elink_trailer = trailers[trailers['elink'] == elink]
            chip_id = int(elink_trailer['chipid'].iloc[0]) if not elink_trailer.empty else -1
        else:
            chip_id = -1 # No trailer found for this event at all
        
        all_hits['row_id'].extend(elink_hits['row_id'].tolist())
        all_hits['col_id'].extend(elink_hits['col_id'].tolist())
        all_hits['toa_code'].extend(elink_hits['toa'].tolist())
        all_hits['tot_code'].extend(elink_hits['tot'].tolist())
        all_hits['cal_code'].extend(elink_hits['cal'].tolist())
        all_hits['elink'].extend(elink_hits['elink'].tolist())
        all_hits['chip_id'].extend([chip_id] * len(elink_hits))

    return {
        'bcid': int(headers['bcid'].iloc[0]) if not headers.empty else -1,
        'l1counters': headers['l1counter'].tolist(),
        'nhits': len(all_hits['row_id']),
        **all_hits,
    }

def process_and_merge_dat_file(input_dat_file, bcid_window=10):
    output_json_file = os.path.splitext(input_dat_file)[0] + "_merged.json"
    
    print(f"\n-> Reading raw data from: {input_dat_file}")
    df_decoder = DataFrame('ETROC2')

    try:
        with open(input_dat_file, 'rb') as f:
            bin_data = f.read()
            raw_data = struct.unpack(f'<{int(len(bin_data)/4)}I', bin_data)
    except FileNotFoundError:
        print(red(f"ERROR: Input file not found at '{input_dat_file}'"))
        return None, None

    merged_data = merge_words(raw_data)
    unpacked_data = [df_decoder.read(x) for x in merged_data]
    print(f"-> Unpacked {len(unpacked_data)} data words.")

    events, current_event_fragments = [], []
    for data_type, data_dict in unpacked_data:
        if data_type == 'header':
            if not current_event_fragments:
                current_event_fragments.append(data_dict)
            else:
                bcid_diff = abs(current_event_fragments[0]['bcid'] - data_dict['bcid'])
                bcid_diff = min(bcid_diff, 3564 - bcid_diff)
                if bcid_diff < bcid_window:
                    current_event_fragments.append(data_dict)
                else:
                    events.append(_aggregate_fragments(current_event_fragments))
                    current_event_fragments = [data_dict]
        elif current_event_fragments:
            current_event_fragments.append(data_dict)
    if current_event_fragments:
        events.append(_aggregate_fragments(current_event_fragments))

    events_ak = ak.Array(events)
    print(green(f"-> Successfully merged into {len(events_ak)} events."))

    with open(output_json_file, "w") as f:
        f.write(ak.to_json(events_ak))
    print(green(f"✅ Final merged data saved to: {output_json_file}"))
    
    return events_ak, output_json_file

# if 'output_dat_file' in locals() and os.path.exists(output_dat_file):
#     output_json_file = os.path.splitext(output_dat_file)[0] + "_merged.json"
#     process_and_merge_by_bcid(output_dat_file, output_json_file)
# else:
#     print(red("Error: Could not find the `output_dat_file` variable or the file itself."))
#     print(red("Please make sure you have run the previous data acquisition cell successfully."))

In [ ]:
if output_dat_file:
    final_events, saved_json_path = process_and_merge_dat_file(output_dat_file)

    if final_events is not None:
        print("\n--- First 5 Merged Events from output file ---")
        for i, event in enumerate(final_events[:5]):
            print(f"\nEvent {i} (BCID: {event['bcid']}):")
            print(f"  L1A Counters from modules: {event['l1counters']}")
            print(f"  Total Hits in event: {event['nhits']}")
            for j in range(event['nhits']):
                 print(f"    - Hit {j+1}: [Elink: {event['elink'][j]}, Chip ID: {event['chip_id'][j]}, Row: {event['row_id'][j]}, Col: {event['col_id'][j]}, ToT: {event['tot_code'][j]}]")

